## **Uploading My DataSets**

In [2]:
from google.colab import files
uploaded = files.upload()  # upload all Olist CSVs you plan to use

Saving olist_orders_dataset.csv to olist_orders_dataset.csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset.csv
Saving olist_order_payments_dataset.csv to olist_order_payments_dataset.csv
Saving olist_order_reviews_dataset.csv to olist_order_reviews_dataset.csv
Saving olist_products_dataset.csv to olist_products_dataset.csv
Saving olist_customers_dataset.csv to olist_customers_dataset.csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset.csv
Saving olist_geolocation_dataset.csv to olist_geolocation_dataset.csv


# **Loading in the Datasets**

In [3]:
import pandas as pd

orders      = pd.read_csv('olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'])
items       = pd.read_csv('olist_order_items_dataset.csv')
payments    = pd.read_csv('olist_order_payments_dataset.csv')
reviews     = pd.read_csv('olist_order_reviews_dataset.csv', parse_dates=['review_creation_date','review_answer_timestamp'])
customers   = pd.read_csv('olist_customers_dataset.csv')
products    = pd.read_csv('olist_products_dataset.csv')
sellers     = pd.read_csv('olist_sellers_dataset.csv')
# geolocation is optional for first version

# **Combining all the Datasets into 1 Dataframe**

In [4]:
for name, df in [('orders',orders),('items',items),('payments',payments),('reviews',reviews),('customers',customers),('products',products),('sellers',sellers)]:
    print(name, df.shape); print(df.isna().mean().sort_values(ascending=False).head(5), '\n')

orders (99441, 8)
order_delivered_customer_date    0.029817
order_delivered_carrier_date     0.017930
order_approved_at                0.001609
order_id                         0.000000
order_purchase_timestamp         0.000000
dtype: float64 

items (112650, 7)
order_id               0.0
order_item_id          0.0
product_id             0.0
seller_id              0.0
shipping_limit_date    0.0
dtype: float64 

payments (103886, 5)
order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64 

reviews (99224, 7)
review_comment_title      0.883415
review_comment_message    0.587025
review_id                 0.000000
review_score              0.000000
order_id                  0.000000
dtype: float64 

customers (99441, 5)
customer_id                 0.0
customer_unique_id          0.0
customer_zip_code_prefix    0.0
customer_city               0.0
customer_state              0.0
dtype: float64 

# **Cleaning the Data**

In [5]:
# Order monetary value at order_id level
gmv_by_order = items.groupby('order_id').agg(
    gmv=('price','sum'),
    freight_value=('freight_value','sum'),
    total_items=('order_item_id','count')
).reset_index()

# Payment summaries per order
pay_by_order = payments.groupby('order_id').agg(
    payment_value=('payment_value','sum'),
    num_payments=('payment_sequential','max'),
    main_payment_type=('payment_type', lambda s: s.value_counts().idxmax())
).reset_index()

# Review score (NPS-ish proxy)
rev_by_order = reviews.groupby('order_id').agg(
    review_score=('review_score','mean')
).reset_index()

# Customer base table
cust = customers[['customer_id','customer_unique_id','customer_city','customer_state']]

# Orders trimmed + delivery KPIs
ordx = orders[['order_id','customer_id','order_status','order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']]
ordx['delivered_late'] = (ordx['order_delivered_customer_date'] > ordx['order_estimated_delivery_date']).astype('Int64')
ordx['delivery_days'] = (ordx['order_delivered_customer_date'] - ordx['order_purchase_timestamp']).dt.days
ordx['order_date'] = ordx['order_purchase_timestamp'].dt.date
ordx['order_ym'] = ordx['order_purchase_timestamp'].dt.to_period('M').astype(str)

/tmp/ipython-input-1535698798.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordx['delivered_late'] = (ordx['order_delivered_customer_date'] > ordx['order_estimated_delivery_date']).astype('Int64')


In [6]:
order_fact = (ordx
  .merge(gmv_by_order, on='order_id', how='left')
  .merge(pay_by_order, on='order_id', how='left')
  .merge(rev_by_order, on='order_id', how='left')
  .merge(cust, on='customer_id', how='left')
)

# Cleanups
num_cols = ['gmv','freight_value','payment_value','review_score','delivery_days']
for c in num_cols:
    order_fact[c] = pd.to_numeric(order_fact[c], errors='coerce')

# Simple NA handling
order_fact['review_score'] = order_fact['review_score'].fillna(order_fact['review_score'].median())
order_fact['delivery_days'] = order_fact['delivery_days'].fillna(order_fact['delivery_days'].median())
order_fact['gmv'] = order_fact['gmv'].fillna(0)
order_fact['payment_value'] = order_fact['payment_value'].fillna(order_fact['gmv'])

# AOV & margin-ish proxy (Olist doesn’t give cost; we’ll display GMV & freight)
order_fact['aov'] = order_fact['gmv'] / order_fact.groupby('order_id')['order_id'].transform('count')  # = gmv (since 1 row/order)

order_fact.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivered_late,delivery_days,order_date,order_ym,...,freight_value,total_items,payment_value,num_payments,main_payment_type,review_score,customer_unique_id,customer_city,customer_state,aov
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,0,8.0,2017-10-02,2017-10,...,8.72,1.0,38.71,3.0,voucher,4.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,0,13.0,2018-07-24,2018-07,...,22.76,1.0,141.46,1.0,boleto,4.0,af07308b275d755c9edb36a90c618231,barreiras,BA,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,0,9.0,2018-08-08,2018-08,...,19.22,1.0,179.12,1.0,credit_card,5.0,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,0,13.0,2017-11-18,2017-11,...,27.20,1.0,72.20,1.0,credit_card,5.0,7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,RN,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,0,2.0,2018-02-13,2018-02,...,8.72,1.0,28.62,1.0,credit_card,5.0,72632f0f9dd73dfee390c9b22eb56dd6,santo andre,SP,19.90


In [7]:
df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


# **Exporting the Data**

In [8]:
order_fact.to_csv('olist_orders_fact_clean.csv', index=False)

from google.colab import files
files.download('olist_orders_fact_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>